In [3]:
from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorOphysProjectCache
import visual_behavior.database as db

In [12]:
def get_cell_table(ophys_session_ids=None, columns_to_return='*'):
    '''
    retrieves the full cell_specimen table from LIMS for the specified ophys_experiment_ids
    if no ophys_experiment_ids are passed, all experiments from the `VisualBehaviorOphysProjectCache` will be retrieved
    
    Parameters
    ----------
    ophys_session_ids : list
        A list of ophys_experiment_ids for which to retrieve the associated cells.  
        If None, all experiments from the `VisualBehaviorOphysProjectCache` will be retrieved.
        Default = None
    columns_to_return
        A list of which colums to return. 
        If "*" is passed, all columns will be returned.
        Queries will be faster if fewer columns are returned.
        Possible columns that can be returned:
            cell_roi_id
            cell_specimen_id
            ophys_experiment_id
            x
            y
            width
            height
            valid_roi
            mask_matrix
            max_correction_up
            max_correction_down
            max_correction_right
            max_correction_left
            mask_image_plane
            ophys_cell_segmentation_run_id
        default = '*'
        
    Returns
    -------
    pandas.DataFrame
        A dataframe with one row per cell and each of the requested columns
        
        
    Examples:
    -------
    This will return all columns for all released experiments
    This takes about 5 seconds
    >> cell_table = get_cell_table()
    
    This will return only the columns ['ophys_experiment_id','cell_specimen_id'] for all released experiments
    This takes about 1.5 seconds
    >> get_cell_table(columns_to_return = ['ophys_experiment_id','cell_specimen_id'])
    
    This will return only the columns ['ophys_experiment_id','cell_specimen_id'] for the specified experiments
    This takes about 20 ms
    >> oeids = [792813858, 888876943, 986518885, 942596355, 908381680]
    >> cell_table = get_cell_table(ophys_session_ids = oeids, columns_to_return = ['ophys_experiment_id','cell_specimen_id'])
    
    This will return all columns for the specified experiments
    This takes about 50 ms
    >> oeids = [792813858, 888876943, 986518885, 942596355, 908381680]
    >> cell_table = get_cell_table(ophys_session_ids = oeids)
    
    
    '''
    # get ophys_session_ids from S3 if they were not passed
    if ophys_session_ids is None:
        data_storage_directory = '/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/production_cache'
        cache = VisualBehaviorOphysProjectCache.from_s3_cache(cache_dir=data_storage_directory)

        experiment_table = cache.get_ophys_experiment_table().reset_index()

        ophys_session_ids = experiment_table['ophys_experiment_id'].unique()
        
        
    if columns_to_return != '*':
        columns_to_return = ', '.join(columns_to_return)
        
    query = '''
        select {}
        from cell_rois 
        where ophys_experiment_id in {} and cell_specimen_id is not null and valid_roi = True
    '''
    
    # Since we are querying from the 'cell_rois' table, the 'id' column is actually 'cell_roi_id'. Rename.
    lims_rois = db.lims_query(
        query.format(columns_to_return, tuple(ophys_session_ids))
    ).rename(columns = {'id':'cell_roi_id'})
    
    return lims_rois

In [7]:
%%time
get_cell_table(columns_to_return = ['ophys_experiment_id','cell_specimen_id'])

CPU times: user 374 ms, sys: 43.4 ms, total: 417 ms
Wall time: 1.27 s


,ophys_experiment_id,cell_specimen_id
0,775614751,1086496928
1,775614751,1086496914
2,775614751,1086496838
3,775614751,1086491756
4,775614751,1086491699
...,...,...
92204,1071440875,1086631138
92205,1071440875,1086631077
92206,1071440875,1086630685
92207,1071440875,1086630543


In [8]:
%%time
get_cell_table()

CPU times: user 3.57 s, sys: 422 ms, total: 4 s
Wall time: 5.39 s


,cell_roi_id,cell_specimen_id,ophys_experiment_id,x,y,width,height,valid_roi,mask_matrix,max_correction_up,max_correction_down,max_correction_right,max_correction_left,mask_image_plane,ophys_cell_segmentation_run_id
0,1080884343,1086496928,775614751,63,380,17,19,True,"[[False, False, False, False, False, True, Tru...",13.0,16.0,8.0,23.0,1,1080818871
1,1080884173,1086496914,775614751,272,463,19,15,True,"[[False, False, False, False, False, False, Fa...",13.0,16.0,8.0,23.0,0,1080818871
2,1080883843,1086496838,775614751,210,361,17,17,True,"[[False, False, False, False, False, False, Fa...",13.0,16.0,8.0,23.0,0,1080818871
3,1080886674,1086491756,775614751,136,230,15,17,True,"[[False, False, False, False, False, False, Fa...",13.0,16.0,8.0,23.0,0,1080818871
4,1080885658,1086491699,775614751,45,435,14,20,True,"[[True, False, False, False, False, False, Fal...",13.0,16.0,8.0,23.0,0,1080818871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92204,1080639610,1086631138,1071440875,252,421,13,14,True,"[[False, False, False, True, True, True, False...",20.0,10.0,10.0,0.0,0,1080628312
92205,1080639591,1086631077,1071440875,53,422,12,18,True,"[[False, False, False, True, True, True, False...",20.0,10.0,10.0,0.0,0,1080628312
92206,1080639763,1086630685,1071440875,69,224,15,27,True,"[[False, False, True, True, False, False, Fals...",20.0,10.0,10.0,0.0,0,1080628312
92207,1080639658,1086630543,1071440875,169,322,15,13,True,"[[False, False, False, False, True, True, True...",20.0,10.0,10.0,0.0,0,1080628312


In [9]:
%%time
oeids = [792813858, 888876943, 986518885, 972233187, 914107595, 984914588, 995622557, 993864050, 942596355, 908381680]
get_cell_table(ophys_session_ids = oeids, columns_to_return = ['ophys_experiment_id','cell_specimen_id'])

CPU times: user 5.37 ms, sys: 0 ns, total: 5.37 ms
Wall time: 20.3 ms


,ophys_experiment_id,cell_specimen_id
0,792813858,1086562145
1,792813858,1086561765
2,792813858,1086561729
3,792813858,1086560888
4,792813858,1086560824
...,...,...
518,995622557,1086498820
519,995622557,1086498071
520,995622557,1086497908
521,995622557,1086497129


In [11]:
%%time
oeids = [792813858, 888876943, 986518885, 972233187, 914107595, 984914588, 995622557, 993864050, 942596355, 908381680]
get_cell_table(ophys_session_ids = oeids)

CPU times: user 33.3 ms, sys: 421 µs, total: 33.7 ms
Wall time: 56.4 ms


,cell_roi_id,cell_specimen_id,ophys_experiment_id,x,y,width,height,valid_roi,mask_matrix,max_correction_up,max_correction_down,max_correction_right,max_correction_left,mask_image_plane,ophys_cell_segmentation_run_id
0,1080905964,1086562145,792813858,382,334,13,18,True,"[[False, False, False, False, False, False, Tr...",5.0,18.0,6.0,27.0,0,1080835682
1,1080905864,1086561765,792813858,347,317,15,22,True,"[[False, False, False, False, False, False, Fa...",5.0,18.0,6.0,27.0,1,1080835682
2,1080905797,1086561729,792813858,23,59,22,15,True,"[[False, False, False, False, False, False, Fa...",5.0,18.0,6.0,27.0,0,1080835682
3,1080905332,1086560888,792813858,254,400,14,22,True,"[[False, True, True, False, False, False, Fals...",5.0,18.0,6.0,27.0,0,1080835682
4,1080905317,1086560824,792813858,354,167,23,13,True,"[[False, False, False, False, False, False, Fa...",5.0,18.0,6.0,27.0,1,1080835682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518,1080708488,1086498820,995622557,246,201,21,14,True,"[[False, False, False, False, False, False, Tr...",10.0,8.0,3.0,16.0,0,1080644820
519,1080708463,1086498071,995622557,184,421,18,18,True,"[[False, False, False, False, False, True, Tru...",10.0,8.0,3.0,16.0,0,1080644820
520,1080708459,1086497908,995622557,307,440,19,19,True,"[[True, True, True, False, False, False, False...",10.0,8.0,3.0,16.0,0,1080644820
521,1080708457,1086497129,995622557,419,458,15,19,True,"[[False, False, False, False, False, False, Tr...",10.0,8.0,3.0,16.0,0,1080644820
